# Naive Implementation vs GPU code
This is for debugging purposes only, to see if the GPU implementation is correct or not.

In [1]:
# Imports
import sys
import os
import random
import time
import numpy as np
np.set_printoptions(precision=2, suppress=True)

# the underlying convRBM implementation
sys.path.append(os.path.abspath('../code'))
from convRBM import CRBM
import getData as dataRead

# biopython stuff
#import Bio.SeqIO as sio
#import Bio.motifs.matrix as mat
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
#from Bio import motifs
import math

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '20264' (I am process '24558')
ERROR (theano.sandbox.cuda): Failed to compile cuda_ndarray.cu: libcublas.so.7.0: cannot open shared object file: No such file or directory
ERROR:theano.sandbox.cuda:Failed to compile cuda_ndarray.cu: libcublas.so.7.0: cannot open shared object file: No such file or directory
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: cuda unavilable)


Couldn't import dot_parser, loading of dot files will not be possible.


## Code for the naive cRBM
All formulas are the same as in the writing section of our git repo.
We implement all the basic functions for upward pass, downward pass, prob max pooling, derivative calculation, gibbs sampling and a training procedure.

In [26]:
class NaiveCRBM:

    def __init__ (self, motifLength=1, numMotifs=1, learningRate=0.1, poolingFactor=1):
        self.numberOfKernels = numMotifs
        self.kernelLength = motifLength
        self.poolingFactor = poolingFactor
        self.learningRate = learningRate
        self.setParamsToZero = True
        self.debug = True
        if self.setParamsToZero:
            self.kernels = np.zeros((self.numberOfKernels, 1, 4, self.kernelLength))
            self.bias = np.zeros(self.numberOfKernels)
            self.c = np.zeros(4)
        else:
            self.kernels = np.random.rand(self.numberOfKernels, 1, 4, self.kernelLength)
            self.bias = np.random.rand(self.numberOfKernels)
            self.c = np.random.rand(4)
    
    def setCustomKernels (self, kernels):
        self.numberOfKernels = kernels.shape[0]
        self.kernelLength = kernels.shape[3]
        self.kernels = kernels.astype(float)
        if self.setParamsToZero:
            self.bias = np.zeros(self.numberOfKernels)
        else:
            self.bias = np.random.rand(self.numberOfKernels)

    def initializeMotifs (self):
        pass
        
    def complement (self, kernelSlice):
        return kernelSlice[::-1]

    def forwardBatch (self, data):
        N_h = data.shape[3]-self.kernelLength+1
        H = np.zeros((data.shape[0], self.numberOfKernels, 1, N_h))
        for sample in range(data.shape[0]):
            for k in range(self.numberOfKernels):
                for n in range(N_h):
                    for m in range(self.kernelLength):
                        # calculate the x_i, that is the cross-correlation
                        x = data[sample,0,:,n+m].T.dot(self.kernels[k,0,:,m]) + self.bias[k]
                        #cKernel = self.complement(self.kernels[k,0,:,self.kernelLength-m-1])
                        #x_prime = data[sample,0,:,n+m].T.dot(cKernel) + self.bias[k]
                        H[sample, k, 0, n] += x # + x_prime
        
        if self.debug:
            print "Pre Sigmoid Hidden Layer:"
            print H
        # perform prob max pooling
        P = np.zeros(H.shape)
        S = np.zeros(H.shape)
        H_exp = np.exp(H)
        numBins = N_h / self.poolingFactor
        for sample in range(data.shape[0]):
            for k_pos in range(0, self.numberOfKernels, 2):
                for unit in range(numBins):
                    #print "Doing unit: " + str(unit)
                    # calculate sum within unit
                    sumInUnit = 0
                    for cell in range(self.poolingFactor):
                        curPos = unit*self.poolingFactor+cell
                        sumInUnit += H_exp[sample,k_pos,0,curPos] + H_exp[sample,k_pos+1,0,curPos]
                        
                    # now, calculate the single positions in P
                    arr = []
                    for cell in range(self.poolingFactor):
                        curPos = unit*self.poolingFactor+cell
                        P[sample,k_pos,0,curPos] = H_exp[sample,k_pos,0,curPos] / (sumInUnit + 1)
                        P[sample,k_pos+1,0,curPos] = H_exp[sample,k_pos+1,0,curPos] / (sumInUnit + 1)
                        arr.append(P[sample,k_pos,0,curPos])
                        arr.append(P[sample,k_pos+1,0,curPos])
                    
                    # finally, do the sampling step
                    arr.append(1 / (sumInUnit+1))
                    s = np.random.multinomial(n=1, pvals=np.array(arr),size=1)
                    am = np.argmax(s)
                    if am < self.poolingFactor*2:
                        strand = am % 2
                        pos = unit * self.poolingFactor + (am // 2)
                        #print "Strand: " + str(strand) + " Pos: " + str(pos)
                        S[sample,k_pos+strand,0,pos] = 1
        return [P,S]


    def backwardBatch (self, H):
        
        # calculate full convolution (not valid, therefore padding is applied with zeros)
        N_v = H.shape[3] + self.kernelLength - 1
        pad = self.kernelLength-1
        V = np.zeros((H.shape[0],1,4,N_v))
        Y = np.zeros(V.shape)
        H_pad = np.pad(H,[(0,0),(0,0),(0,0),(pad, pad)], 'constant',constant_values=(0,0))
        for sample in range(H.shape[0]):
            for k in range(self.numberOfKernels):
                for n in range(N_v):
                    for m in range(self.kernelLength):
                        Y[sample,0,:,n] += self.kernels[k,0,:,m] * H_pad[sample,k,0,pad+n-m]
                        
        # calculate softmax on convolved data
        P_V = self.softmax(Y)
        return P_V
        
    def sampleVisibleLayer(self, P_V):
        # sample the visible layer from probabilities
        V = np.zeros(P_V.shape)
        for sample in range(P_V.shape[0]):
            for col in range(P_V.shape[3]):
                V[sample,0,:,col] = np.random.multinomial(n=1,pvals=P_V[sample,0,:,col],size=1)
        return V

    def expectedDerivative (self, H, data):
        G = np.zeros(self.kernels.shape)
        for sample in range(data.shape[0]):
            for k in range(self.numberOfKernels):
                for n_h in range(H.shape[3]):
                    for m in range(self.kernelLength):
                        G[k,0,:,m] += data[sample,0,:,n_h+m] * H[sample,k,0,n_h]

        der_bias = np.mean(np.sum(H, axis=3), axis=0).reshape(-1)
        der_c = np.mean(np.sum(data, axis=3), axis=0).reshape(-1)
        return [G, der_bias, der_c]
    
    def train_model (self, D, numOfCDs):
        # calculate the data gradient for weights (motifs) and bias
        [H_data, S_data] = self.forwardBatch(D)
        if self.debug:
            print "Hidden Layer Probabilities:"
            print H_data
            print "Hidden Layer Sample:"
            print S_data

        # calculate data gradients
        [G_motif_data, G_bias_data, G_c_data] = self.expectedDerivative(H_data, D)

        if self.debug:
            print "Data gradient for motifs"
            print G_motif_data

        # calculate model probs
        S_H = S_data
        for i in range(numOfCDs):
            V_model = self.backwardBatch(S_H)
            S_V = self.sampleVisibleLayer(V_model)
            if self.debug:
                print "Visible Sample for CD " + str(i)
                print S_V
            [H_model, S_H] = self.forwardBatch(S_V)
        
        # compute the model gradients
        [G_motif_model, G_bias_model, G_c_model] = self.expectedDerivative(H_model, S_V)
        
        if self.debug:
            print "Model gradient for motifs:"
            print G_motif_model
        
        # update the parameters
        self.kernels += self.learningRate * (G_motif_data - G_motif_model)
        self.bias += self.learningRate * (G_bias_data - G_bias_model)
        self.c += self.learningRate * (G_c_data - G_c_model)

        
    def trainMinibatch (self, trainData, epochs, batchSize, numOfCDs):
        iterations = trainData.shape[0] / batchSize
        for epoch in range(epochs):
            for batchIdx in range(iterations):
                self.train_model(trainData[batchIdx*batchSize:(batchIdx+1)*batchSize], numOfCDs)
        
    def softmax (self, x):
        return np.exp(x) / np.exp(x).sum(axis=2, keepdims=True)

## Construct toy data to test the code

In [36]:
kernel1 = np.tile(np.array([[1,0,0],[0,1,0],[0,0,1],[0,0,0]]), [1,1,1])
kernel1_ = np.tile(np.flipud(np.fliplr(kernel1[0])),[1,1,1])
kernel2 = np.tile(np.array([[0,0,0],[0,0,0],[1,1,1],[0,0,0]]), [1,1,1])
kernel2_ = np.tile(np.flipud(np.fliplr(kernel2[0])), [1,1,1])
kernel3 = np.random.rand(1,4,3)
kernel3_ = np.tile(np.flipud(np.fliplr(kernel3[0])), [1,1,1])
kernel = np.array([kernel1, kernel1_])#, kernel2, kernel2_])#, kernel3, kernel3_])
#kernel = np.array([kernel3, kernel3_])
print "Kernel: " + str(kernel)

# initialize the data
randSeq1 = dataRead.getOneHotSeq(Seq("ACGTGGGG", IUPAC.unambiguous_dna))
randSeq2 = dataRead.getOneHotSeq(Seq("ACGTACGT", IUPAC.unambiguous_dna))
data = np.array([randSeq1], dtype=np.float32)
print "Data shape: " + str(data.shape)
print data

#initialize the learner and set custom kernels
hyper_params = {'number_of_motifs':1,
                'motif_length':3,
                'learning_rate':0.1,
                'pooling_factor':2
}
naiveModel = NaiveCRBM(motifLength=hyper_params['motif_length'],
                       numMotifs=hyper_params['number_of_motifs'],
                       learningRate=hyper_params['learning_rate'],
                       poolingFactor=hyper_params['pooling_factor'])

gpuModel = CRBM(hyper_params)
gpuModel.setToZero = True
# set parameters
naiveModel.setCustomKernels(kernel)
gpuModel.setCustomKernels(kernel)
gpuModel.batchSize = 1
print gpuModel.printHyperParams()
gpuModel.debug = False
naiveModel.debug = False

Kernel: [[[[1 0 0]
   [0 1 0]
   [0 0 1]
   [0 0 0]]]


 [[[0 0 0]
   [1 0 0]
   [0 1 0]
   [0 0 1]]]]
Data shape: (1, 1, 4, 8)
[[[[ 1.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  1.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  1.  0.  1.  1.  1.  1.]
   [ 0.  0.  0.  1.  0.  0.  0.  0.]]]]
New motifs set. # Motifs: 1 K-mer-Length: 3
{'learning_rate': 0.1,
 'motif_length': 3,
 'number_of_motifs': 1,
 'pooling_factor': 2}
None


In [50]:
seqReader = dataRead.SeqReader()
allSeqs = seqReader.readSequencesFromFile('../data/wgEncodeAwgDnaseUwAg10803UniPk.fa')
realData = np.array([allSeqs[random.randrange(0, len(allSeqs))]])
print realData.shape

skip sequence containing N
(1, 1, 4, 150)


## Perform test of both, the GPU and Naive variant of the code
Test scenarios are the following:
* **Upward pass**
* **Downward pass**
* **Calculation of Derivatives**

In [51]:
import theano
import theano.tensor as T
import theano.tensor.nnet.conv as conv

# create theano functions
# forward
print "Compiliing theano functions..."
D = T.tensor4('data')
[P,S] = gpuModel.forwardBatch(D)
forward = theano.function([D], [P,S], allow_input_downcast=True)

# backward
H = T.tensor4('Hidden')
V = gpuModel.backwardBatch(H)
backward = theano.function([H], V, allow_input_downcast=True)

# gradient
H = T.tensor4('Hidden Probabilities')
D = T.tensor4('Data')
G_m,G_b,G_c = gpuModel.expectedDerivative(H,D)
gradient = theano.function([H,D], [G_m,G_b,G_c], allow_input_downcast=True)

# gibbs sampler (up, down, sample)
D = T.tensor4('data')
[P,S] = gpuModel.forwardBatch(D)
V_P = gpuModel.backwardBatch(S)
V = gpuModel.sampleVisibleLayer(V_P)
gibbs = theano.function([D], V, allow_input_downcast=True)

print "Starting forward pass test:"
print "----------------------------"
[P_naive,S_n] = naiveModel.forwardBatch(data)
print "Done with naive Model"
[P_GPU,S_g] = forward(data)
#print "Naive Result:"
#print P_naive
#print "GPU result:"
#print P_GPU
print "ERROR MADE: " + str(np.sum(np.abs(P_naive-P_GPU)))
print
print "Starting backward pass test:"
print "----------------------------"
print "Hidden Sample: "
V_naive = naiveModel.backwardBatch(S_n)
V_gpu = backward(S_n)
#print "Naive Result (Backward)"
#print V_naive
#print "GPU Result (Backward)"
#print V_gpu
print "ERROR MADE: " + str(np.sum(np.abs(V_naive-V_gpu)))
print
print "Starting Gradient pass test:"
print "----------------------------"
G_M_naive, G_b_naive, G_c_naive = naiveModel.expectedDerivative(P_naive, data)
G_M_gpu,G_b_gpu,G_c_gpu = gradient(P_naive, data)
print "ERROR MADE (Motifs): " + str(np.sum(np.abs(G_M_naive-G_M_gpu)))
print "ERROR MADE (Bias): " + str(np.sum(np.abs(G_b_naive-G_b_gpu)))
print "ERROR MADE (c): " + str(np.sum(np.abs(G_c_naive-G_c_gpu)))

Compiliing theano functions...
Starting forward pass test:
----------------------------
Done with naive Model
ERROR MADE: 3.64856668142e-08

Starting backward pass test:
----------------------------
Hidden Sample: 
ERROR MADE: 6.18965526611e-08

Starting Gradient pass test:
----------------------------
ERROR MADE (Motifs): 9.685754776e-08
ERROR MADE (Bias): 1.01162649813e-07
ERROR MADE (c): 0.0


In [58]:
print "Starting Gibbs Sampling test:"
print "----------------------------"
data = np.array([allSeqs[random.randrange(0,len(allSeqs))] for i in range(10)])
precision = 1000
V_naive_acc = np.zeros(data.shape)
V_gpu_acc = np.zeros(data.shape)
for i in range(precision):
    V_naive = naiveModel.sampleVisibleLayer(naiveModel.backwardBatch(naiveModel.forwardBatch(data)[1]))
    V_gpu = gibbs(data)
    V_naive_acc += V_naive
    V_gpu_acc += V_gpu

V_naive_acc /= precision
V_gpu_acc /= precision
print "ERROR MADE: " + str(np.mean(np.abs(V_naive_acc-V_gpu_acc)))


Starting Gibbs Sampling test:
----------------------------
ERROR MADE: 0.015492


## Compare training procedures of both methods

In [34]:
#data = np.array([allSeqs[random.randrange(0,len(allSeqs))] for i in range(1)])
naiveModel = NaiveCRBM(motifLength=hyper_params['motif_length'],
                       numMotifs=hyper_params['number_of_motifs'],
                       learningRate=hyper_params['learning_rate'],
                       poolingFactor=hyper_params['pooling_factor'])

gpuModel = CRBM(hyper_params)
gpuModel.setToZero = True
# set parameters
naiveModel.setCustomKernels(kernel)
gpuModel.setCustomKernels(kernel)
gpuModel.batchSize = 1
print gpuModel.printHyperParams()
gpuModel.debug = False
naiveModel.debug = False
gpuModel.trainMinibatch(data, 1, 1, 1)
naiveModel.trainMinibatch(data, 1, 1, 1)

New motifs set. # Motifs: 1 K-mer-Length: 3
{'learning_rate': 0.1,
 'motif_length': 3,
 'number_of_motifs': 1,
 'pooling_factor': 2}
None
BatchSize: 1
Num of iterations per epoch: 1
Start compiling Theano training function...
Compilation of Theano training function finished in 1.73778390884 seconds
Start training the model...
Convolution result forward:  __str__ = [[[[ 3.  0.  1.  1.  1.  1.]]

  [[ 0.  3.  0.  1.  1.  1.]]]]
prob max pooled layer:  __str__ = [[[[ 1.  0.  0.  0.  0.  0.]]

  [[ 0.  0.  0.  1.  1.  0.]]]]
Pre sigmoid visible layer:  __str__ = [[[[ 1.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  1.  0.  1.  1.  0.  0.  0.]
   [ 0.  0.  1.  0.  1.  1.  0.  0.]
   [ 0.  0.  0.  0.  0.  1.  1.  0.]]]]
Visible Sample:  __str__ = [[[[ 1.  1.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  1.  0.  1.  0.  0.  0.]
   [ 0.  0.  0.  1.  0.  1.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  1.  1.]]]]
Sample of Visible layer at CD0 __str__ = [[[[ 1.  1.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  1.  0.  1.  0. 

In [33]:
new_motifs_gpu = gpuModel.motifs.get_value()
new_motifs_naive = naiveModel.kernels
print "GPU derivatives"
print new_motifs_gpu
print "Naive Derivatives"
print new_motifs_naive
print np.sum(np.abs(new_motifs_gpu-new_motifs_naive))

GPU derivatives
[[[[ 2.19 -1.42 -0.68]
   [-1.24  2.53 -0.36]
   [-0.78  1.4   3.47]
   [ 1.71 -0.64 -0.56]]]


 [[[-0.27 -0.33 -0.46]
   [ 0.75 -0.12 -0.31]
   [-0.39  0.94  0.31]
   [ 0.29 -0.11  0.85]]]]
Naive Derivatives
[[[[ 1.26 -1.48 -0.88]
   [-1.63  1.79 -0.67]
   [ 0.27  2.15  3.87]
   [ 1.68 -0.9  -0.75]]]


 [[[-0.24 -0.24 -0.3 ]
   [ 0.93 -0.2  -0.3 ]
   [ 0.08  1.31  0.52]
   [-0.13 -0.22  0.73]]]]
7.62626819384


## Test the sampling methods of both implementations
In order to do that, sampling from both implementations has to be done multiple times to ensure the correctness.

In [66]:
precision = 10000
naiveModel.debug = False

print "Start test of sampling for prob max pooling"
print "----------------------------"
[P_naive,S_n] = naiveModel.forwardBatch(data)

S_naive = np.zeros((data.shape[0], naiveModel.numberOfKernels, 1, data.shape[3]-naiveModel.kernelLength+1))
S_GPU = np.zeros(P_naive.shape)
for i in range(precision):
    [P_n,S_n] = naiveModel.forwardBatch(data)
    [P_g,S_g] = forward(data)
    S_naive += S_n
    S_GPU += S_g
S_naive /= precision
S_GPU /= precision

print "ERROR MADE: " + str(np.mean(np.abs(S_naive-S_GPU)))

Start test of sampling for prob max pooling
----------------------------
ERROR MADE: 0.00442577702703


In [61]:
V_P = T.tensor4('Reconstruction Probs')
V = gpuModel.sampleVisibleLayer(V_P)
sampleV = theano.function([V_P], V, allow_input_downcast=True)

precision = 1000
print "Start test of sampling for Visible Layer"
print "----------------------------"
print "V Probs:"
#print V_naive

Sample_naive = np.zeros((data.shape[0], naiveModel.numberOfKernels//2, 4, data.shape[3]))
Sample_gpu = np.zeros(Sample_naive.shape)
for i in range(precision):
    V_n = naiveModel.sampleVisibleLayer(V_naive)
    V_g = sampleV(V_naive)
    Sample_naive += V_n
    Sample_gpu += V_g
Sample_naive /= precision
Sample_gpu /= precision

print "ERROR MADE: " + str(np.mean(np.abs(Sample_naive-Sample_gpu)))

Start test of sampling for Visible Layer
----------------------------
V Probs:
ERROR MADE: 0.0
